In [2]:
import os
from PIL import Image
import torch
from diffusers import StableDiffusionPipeline
from pytorch_pretrained_biggan import BigGAN, one_hot_from_names, truncated_noise_sample
from pytorch_fid.fid_score import calculate_fid_given_paths
from torchvision.utils import save_image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.models import inception_v3
from torchvision import transforms
from scipy.stats import entropy
from scipy.linalg import sqrtm
import numpy as np
import matplotlib.pyplot as plt
from min_dalle import MinDalle
import matplotlib.pyplot as plt

In [3]:
# Directory to save images
output_dir = "./images"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

In [3]:
### Stable Diffusion
# Load the pre-trained Stable Diffusion pipeline
print("Loading Stable Diffusion...")
stable_pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to("cuda")

# Stable Diffusion prompts
stable_prompts = [
    "A large airliner jet flying through the blue sky.",
    "A husky is lying under an elephant toy.",
    "A lady dressed in a blue and purple outfit wearing a hat made of fruit.",
    "Three zebras and other wild animals out in a semi-green field.",
    "Woman enjoying down hill skiing at a well-groomed resort.",
    "A close up of dozens of oranges stacked.",
    "airliner",
    "husky"
]
stable_output_dir = os.path.join(output_dir, "stable_diffusion")
os.makedirs(stable_output_dir, exist_ok=True)
# Generate and save Stable Diffusion images
for idx, prompt in enumerate(stable_prompts):
    image = stable_pipe(prompt).images[0]
    filename = os.path.join(stable_output_dir, f"stable_diffusion_{prompt.replace(' ', '_').replace('.', '')}.png")
    image.save(filename)
    print(f"Stable Diffusion - Saved: {filename}")

Loading Stable Diffusion...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Stable Diffusion - Saved: ./images\stable_diffusion\stable_diffusion_A_large_airliner_jet_flying_through_the_blue_sky.png


  0%|          | 0/50 [00:00<?, ?it/s]

...

Stable Diffusion - Saved: ./images\stable_diffusion\stable_diffusion_A_close_up_of_dozens_of_oranges_stacked.png


  0%|          | 0/50 [00:00<?, ?it/s]

Stable Diffusion - Saved: ./images\stable_diffusion\stable_diffusion_airliner.png


  0%|          | 0/50 [00:00<?, ?it/s]

Stable Diffusion - Saved: ./images\stable_diffusion\stable_diffusion_husky.png


In [4]:
### BigGAN
# Load the pre-trained BigGAN model
print("Loading BigGAN...")
biggan_model = BigGAN.from_pretrained('biggan-deep-256')
biggan_model.eval()
biggan_output_dir = os.path.join(output_dir, "big_GAN")
os.makedirs(biggan_output_dir, exist_ok=True)
# BigGAN prompts
prompts_list = [
    (["airliner"], "A large airliner jet flying through the blue sky."),
    (["husky", "elephant"], "A husky is lying under an elephant toy."),
    (["cowboy hat", "banana"], "A lady dressed in a blue and purple outfit wearing a hat made of fruit."),
    (["zebra"], "Three zebras and other wild animals out in a semi-green field."),
    (["ski"], "Woman enjoying down hill skiing at a well-groomed resort."),
    (["husky"], "husky"),
    (["airliner"], "airliner"),
    (["orange"], "A close up of dozens of oranges stacked.")
]
#Generate a BigGAN image and save it with a filename based on the descriptive prompt
def generate_biggan_image_with_labels_auto_weights(model, output_dir, prompts, descriptive_prompt, weights=None):
    # Check that at least one prompt is provided
    assert len(prompts) > 0, "At least one prompt must be provided."

    # Assign equal weights if none are provided
    if weights is None:
        weights = [1.0 / len(prompts)] * len(prompts)

    # Normalize weights to sum to 1
    total_weight = sum(weights)
    weights = [w / total_weight for w in weights]

    # Generate class vector
    class_vectors = [one_hot_from_names([prompt], batch_size=1) for prompt in prompts]
    class_vectors = [torch.from_numpy(cv).float() for cv in class_vectors if cv is not None]

    if not class_vectors:
        print(f"BigGAN - Invalid prompts: {prompts}. Ensure they are valid ImageNet classes.")
        return

    # Blend class vectors using the normalized weights
    blended_class_vector = sum(w * cv for w, cv in zip(weights, class_vectors))

    # Generate noise vector
    noise_vector = truncated_noise_sample(truncation=0.4, batch_size=1)
    noise_vector = torch.from_numpy(noise_vector).float()

    # Generate the image
    with torch.no_grad():
        output = model(noise_vector, blended_class_vector, truncation=0.4)

    output = (output.clamp(min=-1, max=1) + 1) / 2.0  # Rescale to [0, 1]

    # Prepare a filename based on the descriptive prompt
    sanitized_prompt = descriptive_prompt.replace(" ", "_").replace(".", "")  # Replace spaces and remove dots
    filename = os.path.join(output_dir, f"big_GAN_{sanitized_prompt}.png")

    # Save the image
    save_image(output, filename)
    print(f"BigGAN - Saved blended image: {filename}")



# Process the prompts list and generate images
def process_prompts(prompts_list):
    for labels, description in prompts_list:
        # `labels` is a list of class labels
        print(f"Processing Labels: {labels}")
        print(f"Description: {description}")

        # Generate BigGAN images
        generate_biggan_image_with_labels_auto_weights(
            model=biggan_model,
            output_dir=biggan_output_dir,
            prompts=labels,  # Use labels directly as a list
            descriptive_prompt=description
        )


# Execute the processing
process_prompts(prompts_list)

Loading BigGAN...


C:\Users\hjhjjhg\AppData\Roaming\Python\Python312\site-packages\pytorch_pretrained_biggan\model.py:279: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load

Processing Labels: ['airliner']
Description: A large airliner jet flying through the blue sky.
BigGAN - Saved blended image: ./images\big_GAN\big_GAN_A_large_airliner_jet_flying_through_the_blue_sky.png
Processing Labels: ['husky', 'elephant']
Description: A husky is lying under an elephant toy.
BigGAN - Saved blended image: ./images\big_GAN\big_GAN_A_husky_is_lying_under_an_elephant_toy.png
Processing Labels: ['cowboy hat', 'banana']
Description: A lady dressed in a blue and purple outfit wearing a hat made of fruit.
BigGAN - Saved blended image: ./images\big_GAN\big_GAN_A_lady_dressed_in_a_blue_and_purple_outfit_wearing_a_hat_made_of_fruit.png
Processing Labels: ['zebra']
Description: Three zebras and other wild animals out in a semi-green field.
BigGAN - Saved blended image: ./images\big_GAN\big_GAN_Three_zebras_and_other_wild_animals_out_in_a_semi-green_field.png
Processing Labels: ['ski']
Description: Woman enjoying down hill skiing at a well-groomed resort.
BigGAN - Saved blended

In [5]:
### MinDalle
# Initialize MinDalle model
print("Loading MinDalle...")
min_dalle_model = MinDalle(
    models_root='./pretrained',
    dtype=torch.float32,
    device='cuda',
    is_mega=True,
    is_reusable=True
)
dalle_output_dir = os.path.join(output_dir, "min_DALL-E")
os.makedirs(dalle_output_dir, exist_ok=True)
# MinDalle prompts
min_dalle_prompts = [
    "A large airliner jet flying through the blue sky.",
    "A husky is lying under an elephant toy.",
    "A lady dressed in a blue and purple outfit wearing a hat made of fruit.",
    "Three zebras and other wild animals out in a semi-green field.",
    "Woman enjoying down hill skiing at a well-groomed resort.",
    "A close up of dozens of oranges stacked.",
    "airliner",
    "husky"
]

# Generate and save MinDalle images
for prompt in min_dalle_prompts:
    print(f"Generating image for prompt: {prompt}")
    image = min_dalle_model.generate_image(
        text=prompt,
        seed=-1,
        grid_size=1,
        is_seamless=False,
        temperature=1.0,
        top_k=256,
        supercondition_factor=32,
        is_verbose=False
    )
    filename = os.path.join(dalle_output_dir, f"min_DALL-E_{prompt.replace(' ', '_').replace('.', '')}.png")
    image.save(filename)
    print(f"MinDalle - Saved: {filename}")

Loading MinDalle...
using device cuda
intializing TextTokenizer
initializing DalleBartEncoder


C:\ProgramData\anaconda3\envs\my-env\Lib\site-packages\min_dalle\min_dalle.py:122: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params = torch.load(self.encoder_params_path

initializing DalleBartDecoder


C:\ProgramData\anaconda3\envs\my-env\Lib\site-packages\min_dalle\min_dalle.py:140: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params = torch.load(self.decoder_params_path

initializing VQGanDetokenizer


C:\ProgramData\anaconda3\envs\my-env\Lib\site-packages\min_dalle\min_dalle.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params = torch.load(self.detoker_params_path

Generating image for prompt: A large airliner jet flying through the blue sky.


C:\ProgramData\anaconda3\envs\my-env\Lib\site-packages\min_dalle\min_dalle.py:201: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=self.dtype):
C:\ProgramData\anaconda3\envs\my-env\Lib\site-packages\min_dalle\min_dalle.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=self.dtype):
C:\ProgramData\anaconda3\envs\my-env\Lib\site-packages\min_dalle\min_dalle.py:239: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=self.dtype):
C:\ProgramData\anaconda3\envs\my-env\Lib\site-packages\min_dalle\min_dalle.py:251: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.auto

MinDalle - Saved: ./images\min_DALL-E\min_DALL-E_A_large_airliner_jet_flying_through_the_blue_sky.png
Generating image for prompt: A husky is lying under an elephant toy.
MinDalle - Saved: ./images\min_DALL-E\min_DALL-E_A_husky_is_lying_under_an_elephant_toy.png
Generating image for prompt: A lady dressed in a blue and purple outfit wearing a hat made of fruit.
MinDalle - Saved: ./images\min_DALL-E\min_DALL-E_A_lady_dressed_in_a_blue_and_purple_outfit_wearing_a_hat_made_of_fruit.png
Generating image for prompt: Three zebras and other wild animals out in a semi-green field.
MinDalle - Saved: ./images\min_DALL-E\min_DALL-E_Three_zebras_and_other_wild_animals_out_in_a_semi-green_field.png
Generating image for prompt: Woman enjoying down hill skiing at a well-groomed resort.
MinDalle - Saved: ./images\min_DALL-E\min_DALL-E_Woman_enjoying_down_hill_skiing_at_a_well-groomed_resort.png
Generating image for prompt: A close up of dozens of oranges stacked.
MinDalle - Saved: ./images\min_DALL-E\

In [12]:
# Deine paths and models
generated_images_folder = "./images"
real_images_folder = "./real_images"
models = ["big_GAN", "min_DALL-E", "stable_diffusion"]

In [13]:
# Temporary folders for FID calculations
TEMP_REAL_FOLDER = "./temp_real"
TEMP_GENERATED_FOLDER = "./temp_generated"

# Ensure temporary folders exist
os.makedirs(TEMP_REAL_FOLDER, exist_ok=True)
os.makedirs(TEMP_GENERATED_FOLDER, exist_ok=True)


def calculate_fid_for_prompt(prompt, generated_folder, real_folder, models, device="cuda"):
    fid_scores = []

    # Save real image to the temporary real folder
    real_image_path = os.path.join(real_folder, f"{prompt}.jpg")
    if os.path.exists(real_image_path):
        real_image = Image.open(real_image_path)
        real_image.save(os.path.join(TEMP_REAL_FOLDER, f"{prompt}.jpg"))
    else:
        print(f"Real image for prompt '{prompt}' is missing.")
        return []

    # Calculate FID for each model
    for model_name in models:
        generated_image_path = os.path.join(generated_folder, model_name, f"{model_name}_{prompt}.png")
        if not os.path.exists(generated_image_path):
            generated_image_path = os.path.join(generated_folder, model_name, f"{model_name}_{prompt}.jpg")
        
        if os.path.exists(generated_image_path):
            generated_image = Image.open(generated_image_path)
            generated_image.save(os.path.join(TEMP_GENERATED_FOLDER, f"{model_name}_{prompt}.png"))

            # Calculate FID
            fid_score = calculate_fid_given_paths(
                [TEMP_REAL_FOLDER, TEMP_GENERATED_FOLDER],
                batch_size=10,
                device=device,
                dims=2048
            )
            fid_scores.append((model_name, fid_score))

            # Clean up temporary generated folder
            os.remove(os.path.join(TEMP_GENERATED_FOLDER, f"{model_name}_{prompt}.png"))
        else:
            print(f"Generated image for model '{model_name}' and prompt '{prompt}' is missing.")

    # Clean up temporary real folder
    os.remove(os.path.join(TEMP_REAL_FOLDER, f"{prompt}.jpg"))
    return fid_scores


In [14]:
def plot_combined_images_and_fid(prompt, generated_folder, real_folder, output_folder, fid_scores, models):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Prepare figure: one real image + one for each model
    num_images = len(models) + 1  # +1 for the real image
    fig, axes = plt.subplots(1, num_images, figsize=(6 * num_images, 6))  # Dynamic layout

    # Load the real image
    real_image_path = os.path.join(real_folder, f"{prompt}.jpg")
    if os.path.exists(real_image_path):
        real_image = Image.open(real_image_path)
        axes[0].imshow(real_image)
        axes[0].set_title("Real Image", fontsize=12, pad=10)  # Added padding
        axes[0].axis('off')
    else:
        axes[0].axis('off')
        axes[0].set_title("Real Image (Missing)", fontsize=12, pad=10)  # Added padding

    # Add generated images with FID scores
    for idx, model_name in enumerate(models):
        generated_image_path = os.path.join(generated_folder, model_name, f"{model_name}_{prompt}.png")
        if not os.path.exists(generated_image_path):
            generated_image_path = os.path.join(generated_folder, model_name, f"{model_name}_{prompt}.jpg")
        
        if os.path.exists(generated_image_path):
            generated_image = Image.open(generated_image_path)
            axes[idx + 1].imshow(generated_image)

            # Find corresponding FID score
            fid_score = next((score for name, score in fid_scores if name == model_name), None)
            if fid_score is not None:
                axes[idx + 1].set_title(f"{model_name}\nFID: {fid_score:.2f}", fontsize=10, pad=10)  # Added padding
            else:
                axes[idx + 1].set_title(model_name, fontsize=10, pad=10)  # Added padding
        else:
            axes[idx + 1].axis('off')
            axes[idx + 1].set_title(f"{model_name} (Missing)", fontsize=12, pad=10)  # Added padding
        axes[idx + 1].axis('off')

    # Adjust layout to ensure titles are fully visible
    plt.subplots_adjust(top=0.85)  # Increase space at the top of the figure
    # Save the plot
    output_file_path = os.path.join(output_folder, f"combined_FID_{prompt}.png")
    plt.savefig(output_file_path, dpi=300, bbox_inches="tight")  # Use bbox_inches="tight" to include all elements
    plt.close()
    print(f"Combined plot saved to: {output_file_path}")



In [15]:
def plot_average_fid(avg_fid, output_folder):
    """
    Plot average FID scores for each model with values displayed on bars.

    Args:
    - avg_fid: Dictionary with model names as keys and their average FID scores.
    - output_folder: Path to save the plot.
    """
    os.makedirs(output_folder, exist_ok=True)

    models = list(avg_fid.keys())
    avg_fid_scores = list(avg_fid.values())

    plt.figure(figsize=(8, 6))
    bars = plt.bar(models, avg_fid_scores, color='green')
    plt.title("Average FID Scores per Model")
    plt.ylabel("Average FID Score")
    plt.xlabel("Model")
    plt.xticks(rotation=45)

    # Add text on bars
    for bar, score in zip(bars, avg_fid_scores):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 1,  # Position the text slightly above the bar
            f"{score:.2f}",  # Format the score with 2 decimal places
            ha='center', va='bottom', fontsize=10
        )

    plt.tight_layout()
    output_path = os.path.join(output_folder, "Average_FID_per_model.png")
    plt.savefig(output_path, dpi=300)
    plt.close()
    print(f"Saved average FID plot to: {output_path}")


In [16]:
def process_all_prompts(prompts, generated_folder, real_folder, output_folder, models, device="cuda"):
    # Store average FID scores
    average_fid_scores = {model: [] for model in models}

    for prompt in prompts:
        print(f"Processing prompt: {prompt}")

        # Calculate FID scores
        fid_scores = calculate_fid_for_prompt(prompt, generated_folder, real_folder, models, device=device)

        # Add scores to averages
        for model_name, fid_score in fid_scores:
            average_fid_scores[model_name].append(fid_score)

        # Generate combined plot for this prompt
        plot_combined_images_and_fid(prompt, generated_folder, real_folder, output_folder, fid_scores, models)

    # Calculate and plot average FID scores
    avg_fid = {model: sum(scores) / len(scores) for model, scores in average_fid_scores.items() if scores}
    plot_average_fid(avg_fid, output_folder)


In [17]:
# Example usage
prompts = [
    "A_large_airliner_jet_flying_through_the_blue_sky",
    "A_husky_is_lying_under_an_elephant_toy",
    "A_lady_dressed_in_a_blue_and_purple_outfit_wearing_a_hat_made_of_fruit",
    "Three_zebras_and_other_wild_animals_out_in_a_semi-green_field",
    "Woman_enjoying_down_hill_skiing_at_a_well-groomed_resort",
    "A_close_up_of_dozens_of_oranges_stacked",
    "airliner",
    "husky"
]

output_folder = "./output_combined"

process_all_prompts(prompts, generated_images_folder, real_images_folder, output_folder, models)

Processing prompt: A_large_airliner_jet_flying_through_the_blue_sky


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.06s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.25s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.11s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.01s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.94s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.96s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.96s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.95s/it]


Combined plot saved to: ./output_combined\combined_FID_husky.png
Saved average FID plot to: ./output_combined\Average_FID_per_model.png
